In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
import os
import pandas as pd
from skimage import io, transform
import numpy as np
from torchvision import transforms, utils, datasets
from PIL import Image
import torch.nn.functional as F

In [2]:
trans = transforms.Lambda(lambda x: x.repeat(1, 1, 3) if x.size(2)==1 else x)

In [3]:
x = torch.randn(224, 224, 1)
out = trans(x)
print(out.shape)

torch.Size([224, 224, 3])


In [4]:
class NoneTransform(object):
    ''' Does nothing to the image. To be used instead of None '''
    
    def __call__(self, image):       
        return image

In [5]:
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x),
    ]
))
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x),
    ]
))
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [6]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [7]:
image, label = train_data[0]

In [8]:
image.size()

torch.Size([3, 224, 224])

In [9]:
label

9

In [10]:
"""
Model architecture: Inverted Residual Block (IRB)

Input( b * h * c)    Operator    t( expansion factor / Width multiplier)    c( channels )    s( stride )    n( Repeating layers )

224 ^2 * 3         Conv2d               None                                 32                2                 1

112 ^2 * 32        Bottleneck           1                                    16                1                 1

112 ^2 * 16        Bottleneck           6                                    24                2                 2

56 ^2 * 24         Bottleneck           6                                    32                2                 3

28 ^2 * 32         Bottleneck           6                                    64                2                 4

14 ^2 * 64         Bottleneck           6                                    96                1                 3

14 ^2 * 96         Bottleneck           6                                    160               2                 3

7 ^2 * 160         Bottleneck           6                                    320               1                 1

7 ^2 * 320         Conv2d 1X1           None                                 1280              1                 1

7 ^2 * 1280        avgPool 7*7          None                                  -                -                 -

1 ^2 * 1280        Conv2d 1X1           None                                  k                -                 -

"""

"""
Model : Linear Bottleneck

Input                        Operator                   Output

h * w * k       Conv2d 1*1 , BatchNorm2d, Relu6        h * w * tk

h * w * tk      Depthwise Conv2d 3*3 stride = s , BatchNorm1d, Relu6        h/s * w/s * tk

h/s * w/s * tk  Pointwise Conv2d 1*1 , BatchNorm2d, Relu6                   h/s * w/s * d'

"""

"\nModel : Linear Bottleneck\n\nInput                        Operator                   Output\n\nh * w * k       Conv2d 1*1 , BatchNorm2d, Relu6        h * w * tk\n\nh * w * tk      Depthwise Conv2d 3*3 stride = s , BatchNorm1d, Relu6        h/s * w/s * tk\n\nh/s * w/s * tk  Pointwise Conv2d 1*1 , BatchNorm2d, Relu6                   h/s * w/s * d'\n\n"

In [11]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [12]:
class LinearBottleneck(nn.Module):
  def __init__(self, in_channels, out_channels, stride, expansion, groups, bias=False):
    super(LinearBottleneck, self).__init__()
    self.stride = stride
    self.expansion = expansion
    self.groups = groups
    self.sequential_conv = nn.Sequential(
      nn.Conv2d(in_channels, in_channels * expansion, kernel_size=1, bias=bias, groups=1),
      nn.BatchNorm2d(in_channels * expansion),
      nn.ReLU6(inplace=True),
    )
    self.depthwise_conv = nn.Sequential(
      nn.Conv2d(in_channels * expansion, in_channels * expansion, kernel_size=3, stride=stride, padding=1, bias=bias, groups=groups),
      nn.BatchNorm2d(in_channels * expansion),
      nn.ReLU6(inplace=True),
    )
    self.pointwise_conv = nn.Sequential(
      nn.Conv2d(in_channels * expansion, out_channels, kernel_size=1, bias=bias, groups=1),
      nn.BatchNorm2d(out_channels),
      nn.ReLU6(inplace=True),
    )

  def forward(self, x):
    x = self.sequential_conv(x)
    x = self.depthwise_conv(x)
    x = self.pointwise_conv(x)
    return x

In [13]:
class MobileNetV2(ImageClassificationBase):
  def __init__(self, num_classes=10, width_mult=1.0, groups=8, bias=False):
    super(MobileNetV2, self).__init__()
    self.width_mult = width_mult
    self.groups = groups
    self.bias = bias
    self.first_layer = nn.Sequential(
      nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=bias),
      nn.BatchNorm2d(32),
      nn.ReLU6(inplace=True),
    )
    self.first_bottleneck = nn.Sequential(
      LinearBottleneck(32, 16, 1, 1, groups=32, bias=bias),
    )
    self.second_bottleneck = nn.Sequential(
      LinearBottleneck(16, 24, 2, 6, groups=16, bias=bias),
      LinearBottleneck(24, 24, 1, 6, groups=24, bias=bias),
    )
    self.third_bottleneck = nn.Sequential(
      LinearBottleneck(24, 32, 2, 6, groups=24, bias=bias),
      LinearBottleneck(32, 32, 1, 6, groups=32, bias=bias),
      LinearBottleneck(32, 32, 1, 6, groups=32, bias=bias),
    )
    self.fourth_bottleneck = nn.Sequential(
      LinearBottleneck(32, 64, 2, 6, groups=32, bias=bias),
      LinearBottleneck(64, 64, 1, 6, groups=64, bias=bias),
      LinearBottleneck(64, 64, 1, 6, groups=64, bias=bias),
      LinearBottleneck(64, 64, 1, 6, groups=64, bias=bias),
    )
    self.fifth_bottleneck = nn.Sequential(
      LinearBottleneck(64, 96, 1, 6, groups=64, bias=bias),
      LinearBottleneck(96, 96, 1, 6, groups=96, bias=bias),
      LinearBottleneck(96, 96, 1, 6, groups=96, bias=bias),
    )
    self.sixth_bottleneck = nn.Sequential(
      LinearBottleneck(96, 160, 2, 6, groups=96, bias=bias),
      LinearBottleneck(160, 160, 1, 6, groups=160, bias=bias),
      LinearBottleneck(160, 160, 1, 6, groups=160, bias=bias),
    )
    self.seventh_bottleneck = nn.Sequential(
      LinearBottleneck(160, 320, 1, 6, groups=160, bias=bias),
    )
    self.pointwise_conv = nn.Sequential(
      nn.Conv2d(320, 1280, kernel_size=1, bias=bias),
      nn.BatchNorm2d(1280),
      nn.ReLU6(inplace=True),
    )
    self.avgpool = nn.AvgPool2d(kernel_size=7)
    self.classifier = nn.Linear(1280, num_classes)

  def forward(self, x):
    self.x = x
    x = self.first_layer(x)
    x = self.first_bottleneck(x)
    x = self.second_bottleneck(x)
    x = self.third_bottleneck(x)
    x = self.fourth_bottleneck(x)
    x = self.fifth_bottleneck(x)
    x = self.sixth_bottleneck(x)
    x = self.seventh_bottleneck(x)
    x = self.pointwise_conv(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    return x

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [15]:
model = MobileNetV2()
from torchvision import models
from torchsummary import summary

In [16]:
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]           1,024
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 32, 112, 112]             288
       BatchNorm2d-8         [-1, 32, 112, 112]              64
             ReLU6-9         [-1, 32, 112, 112]               0
           Conv2d-10         [-1, 16, 112, 112]             512
      BatchNorm2d-11         [-1, 16, 112, 112]              32
            ReLU6-12         [-1, 16, 112, 112]               0
 LinearBottleneck-13         [-1, 16, 112, 112]               0
           Conv2d-14         [-1, 96, 1

In [17]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break  

In [18]:
for images, labels in train_loader:
    print('images.shape:', images.shape)
    out = model(images)
    print('out.shape:', out.shape)
    break

images.shape: torch.Size([64, 3, 224, 224])
out.shape: torch.Size([64, 10])


In [19]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [20]:
device = get_default_device()
device

device(type='cpu')

In [21]:
train_loader = DeviceDataLoader(train_loader, device)
test_loader = DeviceDataLoader(test_loader, device)
to_device(model, device);

In [22]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [23]:
model = to_device(MobileNetV2(), device)


In [24]:
evaluate(model, test_loader)

{'val_acc': 0.09982085973024368, 'val_loss': 2.302661180496216}

In [25]:
num_epochs = 5
opt_func = torch.optim.Adam
lr = 0.001

In [ ]:
history = fit(num_epochs, lr, model, train_loader, test_loader, opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')

In [ ]:
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')

In [ ]:
plot_losses(history)

In [ ]:
def predict_image(img, model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return datasets.classes[preds[0].item()]